### Importando as bibliotecas necessárias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import locale
import openpyxl
import re
import warnings
warnings.filterwarnings('ignore')

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#### Carregando o arquivo CSV em um DataFrame
#### Neste passo, carregaremos o arquivo CSV com os dados dos anúncios em um DataFrame do pandas
#### Certifique-se de ter o arquivo 'miro_dados_brutos_desde_janeiro.csv' no mesmo diretório deste notebook

In [2]:
dados_anuncios = pd.read_csv('miro_dados_brutos_desde_janeiro.csv')

In [3]:
# Exibe as primeiras 5 linhas do dataset
print("Primeiras 5 linhas do dataset:")
display(dados_anuncios.head())

Primeiras 5 linhas do dataset:


,Nome do anúncio,Alcance,Impressões,Frequência,Valor usado (BRL),Cliques (todos),CPC (todos),CTR (todos),CPM (custo por 1.000 impressões),Engajamento com a Página,...,Resultados,Custo por resultado,Classificação de qualidade,Contatos por mensagem,Novos contatos de mensagem,CTR único (todos),Custo por clique único (todos),CPC (custo por clique no link),Início dos relatórios,Término dos relatórios
0,AD 01 - VÍDEO STORIES - 04/04/24,42284,172197,4.072391,641.46,109,5.884954,0.063300,3.725152,17748.0,...,25807.0,0.024856,–,NaN,NaN,0.236496,6.414600,128.292000,2024-01-01,2024-11-03
1,AD 01 - CARROSSEL - 18/06/2024,120917,253308,2.094892,640.82,481,1.332266,0.189887,2.529806,262.0,...,120917.0,5.299668,–,2.0,2.0,0.372156,1.424044,2.926119,2024-01-01,2024-11-03
2,AD 01 - CARROSSEL FEED - 05/06/2024,167158,409103,2.447403,562.85,625,0.900560,0.152773,1.375815,381.0,...,167158.0,3.367174,–,1.0,5.0,0.355951,0.945966,1.710790,2024-01-01,2024-11-03
3,AD 01 - VIDEO - 16/09/2024,95498,178060,1.864542,340.81,159,2.143459,0.089296,1.914018,6643.0,...,95498.0,3.568766,Acima da média,NaN,NaN,0.163354,2.184679,170.405000,2024-01-01,2024-11-03
4,AD 01 - VIDEO - 01/10/2024,81964,159631,1.947575,297.95,124,2.402823,0.077679,1.866492,4955.0,...,81964.0,3.635133,Acima da média,NaN,NaN,0.145186,2.503782,148.975000,2024-01-01,2024-11-03


### Verificação de tipos e conversão para tipos mais apropriados

In [4]:
# Exibe tipos de dados de cada coluna
print("\nResumo das colunas e tipos de dados:")
display(dados_anuncios.dtypes)


Resumo das colunas e tipos de dados:


Nome do anúncio                      object
Alcance                               int64
Impressões                            int64
Frequência                          float64
Valor usado (BRL)                   float64
Cliques (todos)                       int64
CPC (todos)                         float64
CTR (todos)                         float64
CPM (custo por 1.000 impressões)    float64
Engajamento com a Página            float64
Conversas por mensagem iniciadas    float64
Reproduções de 25% do vídeo         float64
Reproduções de 50% do vídeo         float64
Reproduções de 75% do vídeo         float64
Reproduções de 95% do vídeo         float64
Reproduções de 100% do vídeo        float64
Orçamento da campanha               float64
Tipo de orçamento da campanha        object
Tipo de resultado                    object
Resultados                          float64
Custo por resultado                 float64
Classificação de qualidade           object
Contatos por mensagem           

In [5]:
# Conversão de tipos de dados
dados_anuncios = dados_anuncios.convert_dtypes()  # Converte colunas para tipos apropriados automaticamente
print("\nTipos de dados após conversão:")
print(dados_anuncios.dtypes)


Tipos de dados após conversão:
Nome do anúncio                     string[python]
Alcance                                      Int64
Impressões                                   Int64
Frequência                                 Float64
Valor usado (BRL)                          Float64
Cliques (todos)                              Int64
CPC (todos)                                Float64
CTR (todos)                                Float64
CPM (custo por 1.000 impressões)           Float64
Engajamento com a Página                     Int64
Conversas por mensagem iniciadas             Int64
Reproduções de 25% do vídeo                  Int64
Reproduções de 50% do vídeo                  Int64
Reproduções de 75% do vídeo                  Int64
Reproduções de 95% do vídeo                  Int64
Reproduções de 100% do vídeo                 Int64
Orçamento da campanha                      Float64
Tipo de orçamento da campanha       string[python]
Tipo de resultado                   string[python]

### Verificação de Valores Ausentes: Adiciona uma análise de valores ausentes por coluna, permitindo identificar colunas que necessitam de tratamento adicional.

In [6]:
#verificação de valores ausentes
print("\nValores ausentes por coluna:")
print(dados_anuncios.isnull().sum())


Valores ausentes por coluna:
Nome do anúncio                      0
Alcance                              0
Impressões                           0
Frequência                           0
Valor usado (BRL)                    0
Cliques (todos)                      0
CPC (todos)                          1
CTR (todos)                          0
CPM (custo por 1.000 impressões)     0
Engajamento com a Página             2
Conversas por mensagem iniciadas    20
Reproduções de 25% do vídeo         23
Reproduções de 50% do vídeo         24
Reproduções de 75% do vídeo         24
Reproduções de 95% do vídeo         24
Reproduções de 100% do vídeo        24
Orçamento da campanha                0
Tipo de orçamento da campanha        0
Tipo de resultado                    8
Resultados                           8
Custo por resultado                  8
Classificação de qualidade           0
Contatos por mensagem               30
Novos contatos de mensagem          22
CTR único (todos)                 

### Removendo linhas duplicadas
### Vamos remover duplicatas, pois elas podem distorcer as métricas e análises dos dados de anúncios

In [7]:
# Identifica e exibe as linhas duplicadas antes de removê-las
duplicatas = dados_anuncios[dados_anuncios.duplicated()]

# Verifica se existem duplicatas e exibe uma mensagem
if not duplicatas.empty:
    print("Linhas duplicadas encontradas e que serão removidas:")
    display(duplicatas)  # Use a função display para exibir o DataFrame no Jupyter Notebook ou em ambientes compatíveis
else:
    print("Nenhuma linha duplicada encontrada.")

# Remove as linhas duplicadas
dados_anuncios = dados_anuncios.drop_duplicates()


Nenhuma linha duplicada encontrada.


### Removendo colunas vazias
### Aqui, removeremos colunas que estejam completamente vazias, pois não contribuem para a análise

In [8]:
# Identifica colunas que estão completamente vazias (apenas NaN)
colunas_vazias = dados_anuncios.columns[dados_anuncios.isna().all()].tolist()

# Verifica se existem colunas vazias e exibe uma mensagem
if colunas_vazias:
    print("As seguintes colunas estão completamente vazias e serão removidas:")
    print(colunas_vazias)
else:
    print("Nenhuma coluna completamente vazia encontrada.")

# Remove as colunas completamente vazias
dados_anuncios = dados_anuncios.dropna(axis=1, how='all')


Nenhuma coluna completamente vazia encontrada.


In [9]:
# Contagem de valores ausentes por coluna
print("\nContagem de valores ausentes por coluna:")
display(dados_anuncios.isnull().sum())


Contagem de valores ausentes por coluna:


Nome do anúncio                      0
Alcance                              0
Impressões                           0
Frequência                           0
Valor usado (BRL)                    0
Cliques (todos)                      0
CPC (todos)                          1
CTR (todos)                          0
CPM (custo por 1.000 impressões)     0
Engajamento com a Página             2
Conversas por mensagem iniciadas    20
Reproduções de 25% do vídeo         23
Reproduções de 50% do vídeo         24
Reproduções de 75% do vídeo         24
Reproduções de 95% do vídeo         24
Reproduções de 100% do vídeo        24
Orçamento da campanha                0
Tipo de orçamento da campanha        0
Tipo de resultado                    8
Resultados                           8
Custo por resultado                  8
Classificação de qualidade           0
Contatos por mensagem               30
Novos contatos de mensagem          22
CTR único (todos)                    1
Custo por clique único (t

In [10]:
# Identificar as linhas onde o valor de 'Alcance' é igual a zero
linhas_excluir = dados_anuncios[dados_anuncios['Alcance'] == 0]

# Extrair os nomes dos anúncios que serão excluídos
anuncios_excluidos = linhas_excluir['Nome do anúncio'].tolist()

# Imprimir os nomes dos anúncios que serão excluídos de forma mais bonita
print("Anúncios excluídos:")
for i, anuncio in enumerate(anuncios_excluidos, start=1):
    print(f"{i}. {anuncio}")

# Excluir as linhas onde 'Alcance' é igual a zero do DataFrame original
dados_anuncios = dados_anuncios[dados_anuncios['Alcance'] != 0]

# Resetar o índice após a exclusão
dados_anuncios.reset_index(drop=True, inplace=True)

Anúncios excluídos:


# Ajuste de tipos das colunas

In [11]:
# Colunas com valores de reproduções de vídeo que podem ser substituídos por 0, onde NaN significa 'sem reproduções'
colunas_reproducoes = [
    "Reproduções de 25% do vídeo",
    "Reproduções de 50% do vídeo",
    "Reproduções de 75% do vídeo",
    "Reproduções de 95% do vídeo",
    "Reproduções de 100% do vídeo",
]

# Substituição de valores NaN por zero
dados_anuncios[colunas_reproducoes] = dados_anuncios[colunas_reproducoes].fillna(0)

In [12]:
dados_anuncios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 29 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Nome do anúncio                   32 non-null     string 
 1   Alcance                           32 non-null     Int64  
 2   Impressões                        32 non-null     Int64  
 3   Frequência                        32 non-null     Float64
 4   Valor usado (BRL)                 32 non-null     Float64
 5   Cliques (todos)                   32 non-null     Int64  
 6   CPC (todos)                       31 non-null     Float64
 7   CTR (todos)                       32 non-null     Float64
 8   CPM (custo por 1.000 impressões)  32 non-null     Float64
 9   Engajamento com a Página          30 non-null     Int64  
 10  Conversas por mensagem iniciadas  12 non-null     Int64  
 11  Reproduções de 25% do vídeo       32 non-null     Int64  
 12  Reproduçõe

In [13]:
dados_anuncios.to_csv("miro_dados_limpos_desde_janeiro.csv", index=False) 